# Emittance measurement simulation

In [ ]:
import sys
import os
from pprint import pprint

import numpy as np
from matplotlib import pyplot as plt
import proplot as pplt
from tqdm import tqdm

sys.path.append('/Users/46h/Research/')
from accphys.tools import beam_analysis as ba
from accphys.tools import plotting as myplt
from accphys.tools import utils

In [ ]:
pplt.rc['grid.alpha'] = 0.05
pplt.rc['axes.grid'] = False
format_kws = dict(ygrid=True, xgrid=False)
savefig_kws = dict(dpi=300)

## Load data

In [ ]:
turns = np.loadtxt('_output/data/turns.dat').astype(int)
print(turns)

In [ ]:
intensities = np.loadtxt('_output/data/intensities.dat')
print(intensities)

In [ ]:
file = open('_output/data/rec_node_names.txt', 'r')
rec_node_names = [line.rstrip() for line in file]
file.close()

In [ ]:
rec_node_positions = np.loadtxt('_output/data/rec_node_positions.dat')

Sigmas_rec[i, j, :, :] is the reconstructed covariance matrix $\Sigma$ for node i, trial j. We also compute the mean and standard deviation of $\Sigma$ across trials. We use the mean values to compute the beam statistics.

In [ ]:
Sigmas_rec_list = []
Sigmas_rec_mean_list = []
Sigmas_rec_std_list = []
for frame in range(len(turns)):
    Sigmas = np.load('_output/data/Sigmas_rec_{}.npy'.format(frame))
    Sigmas_rec_list.append(Sigmas)
    Sigmas_rec_mean_list.append(np.mean(Sigmas, axis=1))
    Sigmas_rec_std_list.append(np.std(Sigmas, axis=1))

In [ ]:
stats_rec_list = []
for Sigmas_mean in tqdm(Sigmas_rec_mean_list):
    moments_mean = np.array([utils.mat2vec(Sigma) for Sigma in Sigmas_mean])
    stats_rec = ba.StatsReader()
    stats_rec.read_moments(moments_mean)
    stats_rec_list.append(stats_rec)

Do the same for the simulated moments. These are calculated directly from the simulated bunch.

In [ ]:
Sigmas_sim_list = []
Sigmas_sim_mean_list = []
Sigmas_sim_std_list = []
for frame in range(len(turns)):
    Sigmas = np.load('_output/data/Sigmas_sim_{}.npy'.format(frame))
    Sigmas_rec_list.append(Sigmas)
    Sigmas_sim_mean_list.append(np.mean(Sigmas, axis=1))
    Sigmas_sim_std_list.append(np.std(Sigmas, axis=1))

In [ ]:
stats_sim_list = []
for Sigmas_mean in tqdm(Sigmas_sim_mean_list):
    moments_mean = np.array([utils.mat2vec(Sigma) for Sigma in Sigmas_mean])
    stats_sim = ba.StatsReader()
    stats_sim.read_moments(moments_mean)
    stats_sim_list.append(stats_sim)

## Comparison 

In [ ]:
line_kws_sim = dict(lw=1.0)
line_kws_rec = dict(ls='dotted', lw=1.0)

In [ ]:
for i in range(len(turns)):
    stats_rec = stats_rec_list[i]
    stats_sim = stats_sim_list[i]
    turn = turns[i]
    
    fig, axes = pplt.subplots(nrows=2, figsize=(5.5, 3.75))
    g1 = axes[0].plot(rec_node_positions[:, 0], stats_sim.twiss2D['beta_x'], **line_kws_sim)
    g2 = axes[0].plot(rec_node_positions[:, 0], stats_sim.twiss2D['beta_y'], **line_kws_sim)
    axes[0].format(cycle='colorblind')
    g3 = axes[0].plot(rec_node_positions[:, 0], stats_rec.twiss2D['beta_x'], **line_kws_rec)
    g4 = axes[0].plot(rec_node_positions[:, 0], stats_rec.twiss2D['beta_y'], **line_kws_rec)
    axes[0].legend(
        [g1, g2, g3, g4], 
        labels=[r'$\beta_x$ (sim)', r'$\beta_y$ (sim)', 
                r'$\beta_x$ (rec)', r'$\beta_y$ (rec)'],
        ncols=1, loc='r', framealpha=0.
    )
    g1 = axes[1].plot(rec_node_positions[:, 0], stats_sim.twiss2D['alpha_x'], **line_kws_sim)
    g2 = axes[1].plot(rec_node_positions[:, 0], stats_sim.twiss2D['alpha_y'], **line_kws_sim)
    axes[1].format(cycle='colorblind')
    g3 = axes[1].plot(rec_node_positions[:, 0], stats_rec.twiss2D['alpha_x'], **line_kws_rec)
    g4 = axes[1].plot(rec_node_positions[:, 0], stats_rec.twiss2D['alpha_y'], **line_kws_rec)
    axes[1].legend(
        [g1, g2, g3, g4], 
        labels=[r'$\alpha_x$ (sim)', r'$\alpha_x$ (sim)', 
                r'$\alpha_x$ (rec)', r'$\alpha_y$ (rec)'],
        ncols=1, loc='r', framealpha=0.
    )
    axes[0].format(
        ylabel='[m/rad]', xlabel='Position [m]', 
        ylim=(0., 105.),
        title='Reconstructed Twiss parameters -- turn {}'.format(turn)
    )
    axes[1].format(
        ylabel='[rad]', xlabel='Position [m]', 
        ylim=(-max(axes[1].get_ylim()), max(axes[1].get_ylim()))
    )
    plt.show()

In [ ]:
for i in range(len(turns)):
    stats_rec = stats_rec_list[i]
    stats_sim = stats_sim_list[i]
    turn = turns[i]
    
    fig, ax = pplt.subplots(figsize=(5.25, 2.0))
    g1 = ax.plot(rec_node_positions[:, 0], stats_sim.twiss2D['eps_x'], **line_kws_sim)
    g2 = ax.plot(rec_node_positions[:, 0], stats_sim.twiss2D['eps_y'], **line_kws_sim)
    g3 = ax.plot(rec_node_positions[:, 0], stats_sim.twiss4D['eps_1'], **line_kws_sim)
    g4 = ax.plot(rec_node_positions[:, 0], stats_sim.twiss4D['eps_2'], **line_kws_sim)
    ax.format(cycle='colorblind')
    g5 = ax.plot(rec_node_positions[:, 0], stats_rec.twiss2D['eps_x'], **line_kws_rec)
    g6 = ax.plot(rec_node_positions[:, 0], stats_rec.twiss2D['eps_y'], **line_kws_rec)
    g7 = ax.plot(rec_node_positions[:, 0], stats_rec.twiss4D['eps_1'], **line_kws_rec)
    g8 = ax.plot(rec_node_positions[:, 0], stats_rec.twiss4D['eps_2'], **line_kws_rec)
    ax.legend(
        handles=[g1, g5, g2, g6, g3, g7, g4, g8], 
        labels=[r'$\varepsilon_x$ (sim)', r'$\varepsilon_x$ (rec)', 
                r'$\varepsilon_y$ (sim)', r'$\varepsilon_y$ (rec)',
                r'$\varepsilon_1$ (sim)', r'$\varepsilon_1$ (rec)', 
                r'$\varepsilon_2$ (sim)', r'$\varepsilon_2$ (rec)',
               ],
        ncols=2, loc='r', framealpha=0.
    )
    ax.format(
        ylabel='[mm mrad]', xlabel='Position [m]', 
        ylim=(0., ax.get_ylim()[1]),
        title='Reconstructed emittances -- turn {}'.format(turn)
    )
    plt.show()

In [ ]:
sim_kws = dict(color='black')
rec_kws = dict(color='red', lw=0, marker='.', ms=3)
# rec_kws = dict(color='red', ls='dotted')
labels = [
    r"$\langle{xx}\rangle$",
    r"$\langle{xx'}\rangle$",
    r"$\langle{xy}\rangle$",
    r"$\langle{xy'}\rangle$",
    r"$\langle{x'x'}\rangle$",
    r"$\langle{x'y}\rangle$",
    r"$\langle{x'y'}\rangle$",
    r"$\langle{yy}\rangle$",
    r"$\langle{yy'}\rangle$",
    r"$\langle{y'y'}\rangle$",
]

In [ ]:
for i in range(len(turns)):
    stats_rec = stats_rec_list[i]
    stats_sim = stats_sim_list[i]
    turn = turns[i]
    fig, axes = pplt.subplots(nrows=10, figsize=(4, 12), spany=False)
    for i, ax in enumerate(axes):
        ax.plot(rec_node_positions[:, 0], stats_sim.moments.iloc[:, i], **sim_kws)
        ax.plot(rec_node_positions[:, 0], stats_rec.moments.iloc[:, i], **rec_kws)
        ax.format(ylabel=labels[i], suptitle='Second-order moments -- turn {}'.format(turn))
    plt.show()

In [ ]:
for i in range(len(turns)):
    stats_rec = stats_rec_list[i]
    stats_sim = stats_sim_list[i]
    turn = turns[i]
    fig, axes = pplt.subplots(nrows=10, figsize=(4, 12), spany=False)
    for i, ax in enumerate(axes):
        ax.plot(rec_node_positions[:, 0], stats_sim.corr.iloc[:, i], **sim_kws)
        ax.plot(rec_node_positions[:, 0], stats_rec.corr.iloc[:, i], **rec_kws)
        ax.format(ylabel=labels[i], suptitle='Correlation coefficients -- turn {}'.format(turn))
    plt.show()

## Wire-scanner accuracy

One worry is that the moments estimated from the wire-scanner profiles will be inaccurate when the transverse beam size is small. We should be able to test this directly.